In [4]:
import streamlit as st
import openai
import langchain
import os
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import Pinecone as langpinecone
from langchain.llms import OpenAI
from pinecone import Pinecone


/Users/weimengng/Documents/VSCode/Navigator/venv/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
from dotenv import load_dotenv
load_dotenv()

# read doc
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents

doc = read_doc('documents/')

def chunk_data(docs, chunk_size=800, chunk_overlap=50):
    text_splitter= RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    doc = text_splitter.split_documents(docs)
    return doc

documents=chunk_data(docs=doc)
len(documents)

1243

In [6]:
## Embedding technique of OpenAI
embeddings = OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
embeddings

vectors=embeddings.embed_query("How are you")
len(vectors)

/Users/weimengng/Documents/VSCode/Navigator/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


1536

In [7]:
# Create a Pinecone instance
pc = Pinecone(
    api_key=os.environ.get('PINECONE_API_KEY'),
    environment="gcp-starter"
)

index_name = "langchainvector"

In [8]:
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name, 
        dimension=1536,  
        metric='cosine'
    )

index = langpinecone.from_documents(doc, embeddings, index_name=index_name)

In [15]:
## Cosine Similiarity Retrieve Results
def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [9]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [10]:
llm=OpenAI(model_name="gpt-3.5-turbo-instruct",temperature=0.5)
chain=load_qa_chain(llm,chain_type="stuff")

/Users/weimengng/Documents/VSCode/Navigator/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [11]:
## Search answers from VectorDB
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [16]:
our_query = "Explain what is a clearing bearing"
answer = retrieve_answers(our_query)
print(answer)

[Document(page_content='BR 45(1)(2)\nPILOTAGE AND BLIND PILOTAGE\n13-24\nOriginal\n1315. Pilotage Planning - Keeping Clear of Dangers\nOnce the  LDL  has been created (see Para 1311d) a nd the tracks selected (see Paras 1312\n/ 1313), Clearing Lines  (Clearing Bearings ) should be plotted to ‘box in’ the safe navigable\nwater completely.   See Para 1316 for Blind Pilotage Clearing Range  techniques.  The definition\nof a  Clearing Line  (Clearing Bearing  / Clearing Range ) is:\n “A Clearing Line  (ie Clearing Bearing / Clearing Range ) is a line over which\nthe Bridge ( Pelorus  Conning position) of a vessel MUST NOT CROSS”.  \na. Offset from Danger.   To allow room for the stern to swing IN from the LDL, while\nturning the bows AWAY, Clearing Lines  should be offset from the  LDL  by a certain\ndistance, proportional to the distance  ‘’  between the Bridge and the stern or bow,l\nwhichever is greater .  The proportion of distance  ‘’  used as an offset from the LDL l\nwill vary with 

/Users/weimengng/Documents/VSCode/Navigator/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 A clearing bearing is a line over which the bridge of a vessel must not cross in order to keep clear of dangers while navigating. It is used in pilotage planning to plot a safe route and ensure the vessel stays within navigable waters. The distance from the bridge to the clearing bearing may vary depending on factors such as the maximum predicted angle of turn away from danger and any gyro or observation errors. In some cases, a reduced offset may be used if the width of the channel is restricted.
